In [10]:

paths = [
    '/data/Pat1/CT_scan/DICOM/PA000001/ST000001/SE000005',
    '/data/Pat2/CT_scan/DICOM/PA000001/ST000001/SE000002',
    '/data/Pat3/CT_scan/DICOM/PA000001/ST000001/SE000002',
    '/data/Pat4/CT_scan/DICOM/PA000001/ST000001/SE000001',
    '/data/Pat5/CT_scan/CAROL_JARVIS_PARKER_DAVID_08_01_2020/Jarvis, Carol KneeR/DICOM/PA000001/ST000001/SE000001',
    '/data/Pat6/CT_scan/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat7/CT_scan/DICOM/PA000001/ST000001/SE000003/',
    '/data/Pat8/CT_scan/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat9/CT_scan/PA000001/ST000001/SE000002/',
    '/data/Pat10/CT_scan/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat11/CT_scan/Maloney,myree/DICOM/PA000001/ST000001/SE000003/',
    '/data/Pat12/CT_scan/STANTON^PETER^^Mr^_CT_22256795E1_360 KS Protocol v9 PreOp Knee(Adult)/Series 003/',
    '/data/Pat13/CT_scan/Wilson, Anthony Knee/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat14/CT_scan/TIJS^THEO^^Mr^_CT_20933520E1_CT Knee - Non Contrast 1112018 (Right)/Series 001/',
    '/data/Pat15/CT_scan/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat16/CT_scan/MARRIOTT^MERLE^^Mrs^_CT_22464345E1_360 KS Protocol v8 Pre Knee(Adult)/Series 002/',
    '/data/Pat17/CT_scan/PA000001/ST000001/SE000002/',
    '/data/Pat18/CT_scan/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat19/CT_scan/DICOM/PA000001/ST000001/SE000001/',
    '/data/Pat20/CT_scan/DICOM/PA000001/ST000001/SE000001/',
]

paths_csv = [
    '/data/Pat1/POD_JR_9060K_Landmarks_femur.csv',
    '/data/Pat2/EVA_JR_9035K_Landmarks_femur.csv',
    '/data/Pat3/ROL_KH_9128K_Landmarks_femur.csv',
    '/data/Pat4/BRO_DL_9146K_Landmarks_femur.csv',
    '/data/Pat5/JAR_DP_9043K_Landmarks_femur.csv',
    '/data/Pat6/TOW_PY_9046K_Landmarks_femur.csv',
    '/data/Pat7/GIL_DP_9140K_Landmarks_femur.csv',
    '/data/Pat8/PRI_ML_9034K_Landmarks_femur.csv',
    '/data/Pat9/MCD_DL_9103K_Landmarks_femur.csv',
    '/data/Pat10/JAC_ML_8976K_Landmarks_femur.csv',
    '/data/Pat11/MAL_ML_8972K_Landmarks_femur.csv',
    '/data/Pat12/STA_AS_8743K_Landmarks_femur.csv',
    '/data/Pat13/WIL_ML_8944K_Landmarks_femur.csv',
    '/data/Pat14/Landmarks_femur.csv',
    '/data/Pat15/HIG_DP_9018K_Landmarks_femur.csv',
    '/data/Pat16/MAR_JB_8722K_Landmarks_femur.csv',
    '/data/Pat17/STE_DL_9085K_Landmarks_femur.csv',
    '/data/Pat18/GRE_DP_9095K_Landmarks_femur.csv',
    '/data/Pat19/GRE_DP_9096K_Landmarks_femur.csv',
    '/data/Pat20/MUR_BF_9127K_Landmarks_femur.csv',
]

output_path = "/code/"

unknown_landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'whitesideReference'
]


In [2]:
import os
import random

import numpy as np
import pandas as pd
import pydicom
from glob import glob
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize

In [3]:

def load_scan(path):
    
    # load the DICOM files
    lstFilesDCM = []  # create an empty list
    for dirName, subdirList, fileList in os.walk(path):
        for filename in fileList:
            if ".zip" not in filename.lower():  # check whether the file's DICOM
                lstFilesDCM.append(os.path.join(dirName,filename))
    files = []
    for fname in lstFilesDCM:
        files.append(pydicom.dcmread(fname))
    slices = []
    skipcount = 0
    for f in files:
        if hasattr(f, 'SliceLocation'):
            slices.append(f)
        else:
            skipcount = skipcount + 1
    slices = sorted(slices, key=lambda s: s.SliceLocation)
    return slices

def get_pixels_hu(slices):
    # create 3D array
    img_shape = list(slices[0].pixel_array.shape)
    img_shape.append(len(slices))
    img3d = np.zeros(img_shape)

    for i, s in enumerate(slices):
        img2d = s.pixel_array
        img3d[:, :, i] = img2d
    img3d[img3d == -2000] = 0
    return img3d


In [4]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float,(list(scan[0].PixelSpacing))+ [scan[0].SliceThickness] )

    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing


In [5]:
def convert_global_aix_to_net_pos(point, spacing, origin):
    point['x'] = (point['x'] - origin[0]) / spacing[0]
    point['y'] = (point['y'] - origin[1]) / spacing[1]
    point['z'] = (point['z'] - origin[2]) / spacing[2]
    return point

In [6]:
# pat_id = 0
# path_csv = paths_csv[pat_id]

# data = pd.read_csv(path_csv, names=["Name", "x", "y", "z"])
# # data = convert_global_aix_to_net_pos(data, spacing, slices[0].ImagePositionPatient)


# cube = 16
# for index, row in data.iterrows():
#     if row['Name'] in unknown_landmarks:
#         print(row['Name'])
#         rad = random.randint(0,cube)
#         print(
#             int(row['x']) - cube - rad, cube + int(row['x']) - rad, ',',
#             int(row['y']) - cube - rad, cube + int(row['y']) - rad, ',',
#             int(row['z']) - cube - rad, cube + int(row['z'] - rad)
#         )
#         label = np.array([int(row['x']), int(row['y']), int(row['z']), rad, rad, rad])
#         print(label)

In [13]:
def main_func(pat_id):
    # pat_id = 0
    path_csv = paths_csv[pat_id]
    data_path = paths[pat_id]
    g = glob(data_path + '/*')
    print("data_path:", data_path)
    print("path_csv:", path_csv)

    slices = load_scan(data_path)
    return slices
    print(len(slices))
    img3d = get_pixels_hu(slices)
    imgs_after_resamp, spacing = resample(img3d, slices, [1,1,1])
    imgs_after_resamp = imgs_after_resamp.transpose(1,0,2)

    data = pd.read_csv(path_csv, names=["Name", "x", "y", "z"])
    data = convert_global_aix_to_net_pos(data, spacing, slices[0].ImagePositionPatient)

#     mark_img = np.zeros(imgs_after_resamp.shape)

#     cude_size = 3
#     for i in range(-cude_size, cude_size):
#         for j in range(-cude_size, cude_size):
#             for k in range(-cude_size, cude_size):
#                 for index, row in data.iterrows():
#                     mark_img[i+int(row['x']), j+int(row['y']), k+int(row['z'])] = 5200

    cube = 16
    for index, row in data.iterrows():
        if row['Name'] in unknown_landmarks:
            print(row['Name'])
            rad = random.randint(0,cube)
#             new_mark = mark_img[
#                 int(row['x']) - cube - rad: cube + int(row['x']) - rad,
#                 int(row['y']) - cube - rad: cube + int(row['y']) - rad,
#                 int(row['z']) - cube - rad: cube + int(row['z']) - rad
#             ]

            new_shape = imgs_after_resamp[
                int(row['x']) - cube - rad: cube + int(row['x']) - rad,
                int(row['y']) - cube - rad: cube + int(row['y']) - rad,
                int(row['z']) - cube - rad: cube + int(row['z']) - rad
            ]
            label = np.array([int(row['x']), int(row['y']), int(row['z']), rad, rad, rad])

            np.save(output_path + row['Name'] + '/images/' + "%d.npy" % (pat_id), new_shape)
            np.save(output_path + row['Name'] + '/label/' + "%d.npy" % (pat_id), label)
    np.save(output_path + 'images/' + "%d.npy" % (pat_id), imgs_after_resamp)
#     np.save(output_path + 'label/' + "%d.npy" % (pat_id), mark_img)


In [14]:
slices = None
for pat_id in range(15):
    slices = main_func(pat_id)
    break

data_path: /data/Pat1/CT_scan/DICOM/PA000001/ST000001/SE000005
path_csv: /data/Pat1/POD_JR_9060K_Landmarks_femur.csv


In [17]:
slices[1]

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0012) Instance Creation Date              DA: '20191104'
(0008, 0013) Instance Creation Time              TM: '160146'
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.840.113619.2.340.3.2831181312.615.1572810739.82.784
(0008, 0020) Study Date                          DA: '20191104'
(0008, 0021) Series Date                         DA: '20191104'
(0008, 0022) Acquisition Date                    DA: '20191104'
(0008, 0023) Content Date                        DA: '20191104'
(0008, 0030) Study Time                          TM: '155657'
(0008, 0031) Series Time                         TM: '155855'
(0008, 0032) Acquisition Time                    TM: '155909.166631'
(0008, 0033) Content Time                        TM: '160146'
(0008, 0050) Accession Number   

In [20]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

import os
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

output_path = "/code/PCLOrigin/"

data_size = 15

imgs_to_process = np.zeros(shape=(data_size, 32, 32, 32))

for index in range(10):
    imgs_to_process[index] = np.load(output_path + 'images/' + '{}.npy'.format(index))

In [23]:
imgs_to_process[0][0][0]

array([1103.38795931, 1036.15976823,  990.72332173,  859.44831657,
        881.0242788 , 1126.78009561, 1369.72906452, 1403.20980239,
       1376.09356172, 1446.11087537, 1430.63413907, 1369.45297514,
       1411.8045319 , 1301.92430502, 1304.6623606 , 1426.59017815,
       1431.67907054, 1287.36982452, 1288.56872192, 1292.3747812 ,
       1251.3233699 , 1264.82468866, 1309.14766976, 1203.29378725,
       1248.54809571, 1292.60142321, 1235.92340303, 1210.32173099,
       1165.40717544, 1247.21082815, 1269.7121789 , 1228.66848597])

In [ ]:
main_func(20-1)

In [ ]:
main_func(18-1)